In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow.keras.callbacks import CSVLogger
import os
import sys
import datetime
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
tfds.disable_progress_bar()
import datetime
#tf.enable_eager_execution()

In [0]:
tf.keras.backend.clear_session()

In [0]:
sys.path.append('/content/drive/My Drive/convAE_2_classifier')
from dataset import Dataset

In [0]:
dataset = Dataset('cats_vs_dogs')
IMG_SIZE = 256
input_shape = (IMG_SIZE, IMG_SIZE, 3)
dataset.set_img_size(IMG_SIZE)
train, val, test = dataset.format_data()

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [0]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000
train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = val.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

In [0]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.DenseNet169(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet',
                                               #weights=None
                                               )
#base_model.load_weights('imagenet')
base_model.trainable = False

In [0]:
for image_batch, label_batch in train_batches.take(1):
   pass

In [0]:
feature_batch = base_model(image_batch)
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
prediction_layer = tf.keras.layers.Dense(1)

In [0]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

In [0]:
base_learning_rate = 0.0001
optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate)
model.compile(optimizer= optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [0]:
file_name = "denseNet169_256"
dateTime = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
csv_path = "./drive/My Drive/convAE_2_classifier/logs/" + file_name +  "_model_history_" + dateTime + ".csv"
csv_logger = CSVLogger(csv_path, separator=',', append=True)
# log_dir="./drive/My Drive/convAE_2_classifier/logs/mobileNet_256X256_logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback  = tf.keras.callbacks.TensorBoard(log_dir=log_dir, update_freq='epoch', histogram_freq=1, write_graph=True, write_images=True)
callbacks =[csv_logger]

In [0]:
initial_epochs = 10
history = model.fit(train_batches,
                    epochs=initial_epochs,
                    validation_data=validation_batches, callbacks=callbacks)

In [0]:
base_model.trainable = True

In [0]:
# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

In [0]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
              metrics=['accuracy'])

In [0]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_batches,
                         epochs=total_epochs,
                         initial_epoch =  history.epoch[-1],
                         validation_data=validation_batches, callbacks=callbacks)

In [0]:
model.save("/content/drive/My Drive/convAE_2_classifier/weights/" + file_name +  "_weights_" + dateTime + ".h5")